In [2]:
from textwrap import wrap
import os

import keras_cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.experimental.numpy as tnp
from keras_cv.models.stable_diffusion.clip_tokenizer import SimpleTokenizer
from keras_cv.models.stable_diffusion.diffusion_model import DiffusionModel
from keras_cv.models.stable_diffusion.image_encoder import ImageEncoder
from keras_cv.models.stable_diffusion.noise_scheduler import NoiseScheduler
from keras_cv.models.stable_diffusion.text_encoder import TextEncoder
from tensorflow import keras

from data_helper import *
from sd_trainer import *
from callbacks import *

import argparse
from datetime import datetime, timezone
import os



In [4]:
test=True
resolution=256
USE_MP=True

In [5]:
training_dataset= get_data_loader(test,resolution)
stable_diffusion_big= keras_cv.models.StableDiffusion(resolution, resolution)
image_encoder=stable_diffusion_big.image_encoder
diffusion_model = stable_diffusion_big.diffusion_model

vae=tf.keras.Model(
    image_encoder.input,
    image_encoder.layers[-2].output,
)

diffusion_ft_trainer = Trainer(
    diffusion_model=diffusion_model,
    vae=vae,
    noise_scheduler= NoiseScheduler(),
    use_mixed_precision=USE_MP
)

Using custom data configuration jlbaker361--little_dataset-9f41739ce432f63f
Found cached dataset parquet (/home/jlb638/.cache/huggingface/datasets/jlbaker361___parquet/jlbaker361--little_dataset-9f41739ce432f63f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [6]:
vae.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 padded_conv2d (PaddedConv2D  (None, 256, 256, 128)    3584      
 )                                                               
                                                                 
 resnet_block (ResnetBlock)  (None, 256, 256, 128)     295680    
                                                                 
 resnet_block_1 (ResnetBlock  (None, 256, 256, 128)    295680    
 )                                                               
                                                                 
 padded_conv2d_5 (PaddedConv  (None, 128, 128, 128)    147584    
 2D)                                                             
                                                             

In [8]:
next(iter(training_dataset))

{'images': <tf.Tensor: shape=(4, 256, 256, 3), dtype=float32, numpy=
 array([[[[-0.40392154, -0.4588235 , -0.5607843 ],
          [-0.41960782, -0.46666664, -0.5607843 ],
          [-0.4352941 , -0.4823529 , -0.5686274 ],
          ...,
          [ 0.41176474,  0.43529415, -0.19999993],
          [ 0.12941182,  0.18431377, -0.41176468],
          [-0.19215679, -0.2156862 , -0.6862745 ]],
 
         [[-0.40392154, -0.45098037, -0.55294114],
          [-0.42745095, -0.4823529 , -0.5686274 ],
          [-0.45098037, -0.4980392 , -0.59999996],
          ...,
          [ 0.3803922 ,  0.4039216 , -0.23921561],
          [ 0.15294123,  0.20784318, -0.41960782],
          [-0.01176465, -0.03529406, -0.57647055]],
 
         [[-0.38823527, -0.4352941 , -0.5215686 ],
          [-0.3960784 , -0.45098037, -0.545098  ],
          [-0.41960782, -0.47450978, -0.5686274 ],
          ...,
          [ 0.38823533,  0.39607847, -0.2156862 ],
          [ 0.254902  ,  0.26274514, -0.3333333 ],
          [ 0

In [13]:
vae(next(iter(training_dataset))["images"])

<tf.Tensor: shape=(4, 32, 32, 8), dtype=float32, numpy=
array([[[[-6.99721336e-01,  1.68603575e+00, -1.73612034e+00, ...,
          -9.26123524e+00, -9.37841415e+00, -9.58389091e+00],
         [-3.05012107e+00, -4.50316238e+00, -1.87621069e+00, ...,
          -1.20447340e+01, -1.20881147e+01, -1.26108246e+01],
         [ 1.46430373e+00,  1.07486308e+00,  6.08134687e-01, ...,
          -1.17901983e+01, -1.16600170e+01, -1.22520494e+01],
         ...,
         [ 3.88032365e+00, -7.02707410e-01, -6.40266120e-01, ...,
          -1.25786448e+01, -1.24268980e+01, -1.33822670e+01],
         [ 4.14370871e+00, -2.80096579e+00,  3.27534080e+00, ...,
          -1.35228701e+01, -1.32043533e+01, -1.46208038e+01],
         [ 3.30716252e+00,  3.77294481e-01,  2.15799284e+00, ...,
          -1.06409369e+01, -1.04794359e+01, -1.13694916e+01]],

        [[-2.21626210e+00,  5.76750278e-01, -2.18763500e-01, ...,
          -1.18515272e+01, -1.19139214e+01, -1.24453974e+01],
         [-1.04281247e+00, -6.82